# Getting Started with BentoML

[BentoML](http://bentoml.ai) is a python framework for building, shipping and running machine learning services. 

This is a quick tutorial on how to use BentoML to define machine learning service with a trained sklearn model, serving via a REST API server and deploy it to AWS Lambda as a serverless endpoint.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=bentoml-quick-start-guide)

In [1]:
# Install BentoML
!pip install bentoml

# Install scikit-learn, we will use a sklean model as an example
!pip install pandas sklearn

Let's get started with a simple scikit-learn model as an example:

In [2]:
from sklearn import svm
from sklearn import datasets

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Define ML Service with BentoML

To package this model with BentoML, you don't need to change anything in your training code. Following your training workflow, create a new BentoML Service by subclassing it:

In [3]:
%%writefile iris_classifier.py
from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

# You can also import your own python module here and BentoML will automatically
# figure out the dependency chain and package all those python modules

@artifacts([PickleArtifact('model')])
@env(pip_dependencies=["scikit-learn"])
class IrisClassifier(BentoService):

    @api(DataframeHandler)
    def predict(self, df):
        # arbitrary preprocessing or feature fetching code can be placed here 
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


The `@artifacts` decorator here tells BentoML what artifacts are required when 
packaging this BentoService. Besides `PickleArtifact`, BentoML also provides
`TfKerasModelArtifact`, `PytorchModelArtifact`, and `TfSavedModelArtifact` etc.

`@env` is designed for specifying the desired system environment in order for this
BentoService to load. If you already have a requirement.txt file listing all python 
libraries you need:
```python
@env(requirement_txt='../myproject/requirement.txt')
```

Lastly `@api` adds an entry point for accessing this BentoService. Each
`api` will be translated into a REST endpoint when [deploying as API
server](#serving-via-rest-api), or a CLI command when [running as a CLI
tool](#use-as-cli-tool).

Each API also requires a `Handler` for defining the expected input format. In
this case, `DataframeHandler` will transform either an HTTP request or CLI
command arguments into a pandas Dataframe and pass it down to the user defined
API function. BentoML also supports `JsonHandler`, `ImageHandler` and
`TensorHandler`.


## Save BentoML service archive

Pack your custom BentoML Service with the trained model:

In [4]:
# 1) import the custom BentoService defined above
from iris_classifier import IrisClassifier

# 2) `pack` it with required artifacts
svc = IrisClassifier.pack(model=clf)

# 3) save packed BentoService as archive
saved_path = svc.save('/tmp/bentoml_archive')

[2019-08-08 20:20:30,017] INFO - Searching for dependant modules of iris_classifier:/Users/leon.sun/bentoml/examples/quick-start/iris_classifier.py
[2019-08-08 20:20:50,748] INFO - Copying local python module '/Users/leon.sun/bentoml/examples/quick-start/iris_classifier.py'
[2019-08-08 20:20:50,750] INFO - Done copying local python dependant modules
[2019-08-08 20:20:50,839] INFO - BentoService IrisClassifier:2019_08_08_e1f4376a saved to /tmp/bentoml_archive/IrisClassifier/2019_08_08_e1f4376a


_That's it._ You've just created your first Bento. It's a versioned archive, containing the BentoService you defined, along with trained model artifacts, dependencies and configurations etc. BentoML library can then load in a Bento file and turn it into a high performance prediction service.

## Serving via REST API

For exposing your model as a HTTP API endpoint, you can simply use the `bentoml
serve` command:

In [ ]:
!bentoml serve {saved_path} --with-conda

Solving environment: done
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Ran pip subprocess with arguments:
['/Users/leon.sun/anaconda3/envs/IrisClassifier_2019_08_08_e1f4376a/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/bentoml_archive/IrisClassifier/2019_08_08_e1f4376a/condaenv.ucnr9t3b.requirements.txt']
Pip subprocess output:
Requirement already up-to-date: bentoml[api_server]==0.3.1 in /Users/leon.sun/anaconda3/envs/IrisClassifier_2019_08_08_e1f4376a/lib/python3.7/site-packages (from -r /tmp/bentoml_archive/IrisClassifier/2019_08_08_e1f4376a/condaenv.ucnr9t3b.requirements.txt (line 1)) (0.3.1)
Requirement already up-to-date: scikit-learn in /Users/leon.sun/anaconda3/envs/IrisClassifier_2019_08_08_e1f4376a/lib/python3.7/site-packages (from -r /tmp/bentoml_archive/IrisClassifier/2019_08_08_e1f4376a/condaenv.ucnr9t3b.requirements.txt (line 2)) (0.21.3)

#
# To activate this environment, use
#
#     $ conda activate IrisClassifier_2019_08

no change     /Users/leon.sun/anaconda3/condabin/conda
no change     /Users/leon.sun/anaconda3/bin/conda
no change     /Users/leon.sun/anaconda3/bin/conda-env
no change     /Users/leon.sun/anaconda3/bin/activate
no change     /Users/leon.sun/anaconda3/bin/deactivate
no change     /Users/leon.sun/anaconda3/etc/profile.d/conda.sh
no change     /Users/leon.sun/anaconda3/etc/fish/conf.d/conda.fish
no change     /Users/leon.sun/anaconda3/shell/condabin/Conda.psm1
no change     /Users/leon.sun/anaconda3/shell/condabin/conda-hook.ps1
no change     /Users/leon.sun/anaconda3/lib/python3.7/site-packages/xonsh/conda.xsh
no change     /Users/leon.sun/anaconda3/etc/profile.d/conda.csh
no change     /Users/leon.sun/.bash_profile
No action taken.
/Users/leon.sun/anaconda3/envs/IrisClassifier_2019_08_08_e1f4376a/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator SVC from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid result

#### View documentations for REST APIs

Open http://127.0.0.1:5000 to view the documentation of generated REST APIs in a 
browser.

*note: RestAPI serving does not work for Google Colab, due to unable to access Colab's port.

#### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[5.1, 3.5, 1.4, 0.2]]' \
localhost:5000/predict
```

Note you must ensure the pip and conda dependencies are available in your python
environment when using `bentoml serve` command. More commonly we recommend using
BentoML API server with Docker:

## Run REST API server with Docker (Optional)

You can build a Docker Image for running API server hosting your BentoML archive
by using the archive folder as docker build context:

__Note that `docker` is note available on Google Colab__

In [ ]:
!cd {saved_path} && docker build -t iris-classifier .

Next, you can `docker push` the image to your choice of registry for deployment,
or run it locally for development and testing:

In [ ]:
!docker run -p 5000:5000 iris-classifier

## Loading BentoService in Python

`bentoml.load` is the enssential API for loading a Bento into your
python application:

In [ ]:
import bentoml
import pandas as pd

bento_svc = bentoml.load(saved_path)

# Test loaded bentoml service:
bento_svc.predict([X[0]])

## "pip install" a BentoML archive

BentoML also supports distributing a BentoService as PyPI package, with the
generated `setup.py` file. A Bento directory can be installed with `pip`:

In [ ]:
!pip install {saved_path}

Now you can import your ML service as a regular python package:

In [ ]:
import IrisClassifier

installed_svc = IrisClassifier.load()
installed_svc.predict([X[0]])

A Bento PyPI package can also be uploaded to pypi.org
as a public python package, or to your organization's private PyPI index for all
developers in your organization to use:

`cd {saved_path} & python setup.py sdist upload`

*You will need a ".pypirc" config file before doing this: https://docs.python.org/2/distutils/packageindex.html*


# CLI access

`pip install {saved_path}` also installs a CLI tool for accessing the BentoML service:

In [ ]:
!IrisClassifier info

In [ ]:
!IrisClassifier docs

In [ ]:
!IrisClassifier --help

In [ ]:
!IrisClassifier predict --help

In [ ]:
!IrisClassifier predict --input='[[5.1, 3.5, 1.4, 0.2]]'

BentoML cli also supports reading input data from `csv` or `json` files, in either local machine or remote HTTP/S3 location:

In [ ]:
# Writing test data to a csv file
pd.DataFrame(iris.data).to_csv('iris_data.csv', index=False)

In [ ]:
# Invoke predict from command lien
!IrisClassifier predict --input='./iris_data.csv'

Alternatively, you can also use the `bentoml` cli to load and run a BentoML service archive without installing it:

In [ ]:
!bentoml info {saved_path}

In [ ]:
!bentoml predict {saved_path} --input='[[5.1, 3.5, 1.4, 0.2]]'

# Deploying to AWS Lambda

AWS Lambda is a serverless computing platform provided by Amazon Web Services. BentoML service archive can be easily deployed to AWS Lambda as a REST API endpoint.

In order to run this demo, make sure to configure your AWS credentials via either `aws configure` command or setting the environment variables below:

In [ ]:
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=

In order to use BentoML for deployment with AWS services, first install extra dependencies that BentoML need:

In [ ]:
!pip install bentoml[aws]

Then install the [serverless framework](https://serverless.com) that BentoML uses under the hood for creating and managing serverless deployments:

In [ ]:
# make sure node is installed
!node --version

# install serverless framework
!npm install -g serverless

Now, you can deploy the BentML service archive you just created to AWS Lambda with one command:

In [ ]:
!bentoml deploy {saved_path} --platform aws-lambda --region us-west-2

To get current deployment status:

In [ ]:
!bentoml check-deployment-status {saved_path} --platform aws-lambda --region us-west-2

And delete your AWS Lambda deployment with:

In [ ]:
!bentoml delete-deployment {saved_path} --platform aws-lambda --region us-west-2

# Summary

This is what it looks like when using BentoML to create and deploy a machine learning service, all the way from training notebook to deployment in production. BentoML also supports many other Machine Learning frameworks, as well as many other deployment platforms. Take a look at other BentoML examples [here](https://github.com/bentoml/BentoML#examples).